#### AI Travel Agent & Expense Planner
##### Purpose: Trip planning for any city worldwide with Realtime data.

In [86]:
from langchain_community.tools import DuckDuckGoSearchResults
from typing import Annotated,TypedDict,List,Sequence
from pydantic import BaseModel,Field
from langchain_core.messages import BaseMessage
import operator

web_tool=DuckDuckGoSearchResults()

In [87]:
web_tool.invoke("Just give temparature in celsius, what is the current temparature at miami beach?")

'snippet: The lowest temperature will be a temperate 80.6°F. Sunrise was at 6:28 am and sunset will be at 8:07 pm; the daylight will last for 13h and 40min. Current condition, title: Weather today - Miami Beach, FL, link: https://www.weather-atlas.com/en/florida-usa/miami-beach, snippet: For comments, questions, daily beach conditions, or more information, please contact the Ocean Rescue Hotline at (305) 673-7714 (option #1 for weather conditions, option #2 to speak with a person) Ocean Rescue Headquarters, title: Current Beach Conditions - City of Miami Beach, link: https://www.miamibeachfl.gov/city-hall/fire/ocean-rescue/currentbeachconditions/, snippet: Current ocean temperature in Miami. Water temperature in Miami today is 84.6°F. Based on our historical data over a period of ten years, the warmest water in this day in Miami was recorded in 2010 and was 86.2°F, and the coldest was recorded in 2007 at 82.4°F. Sea water temperature in Miami is expected to be constant 84.6°F in the ne

In [88]:
class MessageState(TypedDict):
    messages:Annotated[Sequence[BaseMessage], operator.add]

In [3]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from warnings import filterwarnings
filterwarnings("ignore")

load_dotenv()


True

#### `OpenWeatherMap` component to fetch weather information.

In [5]:
import os
from langchain_community.utilities import OpenWeatherMapAPIWrapper

os.environ["OPENWEATHERMAP_API_KEY"] = os.getenv("openweatherkey_one")
weather = OpenWeatherMapAPIWrapper()
weather_data = weather.run("London,GB")
print(weather_data)

In London,GB, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 5.14 m/s, direction: 250°
Humidity: 44%
Temperature: 
  - Current: 25.77°C
  - High: 26.67°C
  - Low: 24.47°C
  - Feels like: 25.55°C
Rain: {}
Heat index: None
Cloud cover: 97%


In [ ]:

from langchain_groq import ChatGroq
from langchain_community.tools import tool
model=ChatGroq(model="llama-3.1-8b-instant")



### Tool to extract Real-time weather information of given Location

In [10]:


@tool
def get_current_weather(Location:str)->str:
    
    "This tool gives current weather of given location"
    weather_data = weather.run(Location)
    System_prompt=["You are a helpful assistant that extract current temparature from given data"]
    query=System_prompt+ [weather_data]
    Result=model.invoke(query)
    return Result.content

get_current_weather.invoke("London,GB")


'The current temperature in London, GB is 25.77°C.'

AI Travel Agent & Expense Planner(Purpose: Trip planning for any city worldwide with Realtime data.")

• Real-time weather information
• Top attractions and activities
• Hotel cost calculation (per day × total days)
• Currency conversion to user's native currency
• Complete itinerary generation
• Total expense calculation
• generate a summary of the entire output

user_input
  |
search attraction and activity
1. search attracation
2. search restaurant
3. search activity
4. search transportation
  |
search weather forcasting
1. get current weather
2. get weather forcast
  |
search hotel costs
1. search hotel
2. estimate the hotel cost
3. budget_range
  |
calculate total cost
1. add
2. multiply
3. calculated total cost
4. calcualte the daily budget
    | 
currency_converion
1. get exchnage rate
2. convert currancy
    | 
Itinery generation
1. get day plan
2. crete full itinery
    |
create Trip Summary
    |
Retun complete traval plan


search attraction and activity
1. search attracation
2. search restaurant
3. search activity
4. search transportation

In [12]:
from langchain_community.tools import tool
search_llm=ChatGroq(model="llama-3.1-8b-instant")

In [20]:

def get_attractions(Location:str)->str:
       
    System_prompt=["You are a helpful assistant that gives list of top 5 attractions, list of top 5 resturants with multi-cusines, list of top 5 activities to do and ways of transporation in given location: "]
    query=System_prompt + [Location]
    Result=search_llm.invoke(query)
    return Result.content

print(get_attractions(Location="London"))

**Welcome to London**

London is a vibrant and diverse city with a rich history, world-class museums, iconic landmarks, and a thriving food scene. Here are some top suggestions for your trip:

**Top 5 Attractions in London:**

1. **Buckingham Palace**: The official residence of the British monarch, Buckingham Palace is a must-visit attraction in London. You can watch the Changing of the Guard ceremony, which takes place daily at 11:30 am from April to July and on alternate days the rest of the year.
2. **The British Museum**: One of the world's greatest museums, the British Museum houses a vast collection of artifacts from ancient civilizations, including the Rosetta Stone, the Elgin Marbles, and the mummies in the Ancient Egypt gallery.
3. **The London Eye**: A giant Ferris wheel on the South Bank of the Thames, the London Eye offers stunning views of the city from its 32 glass-enclosed capsules.
4. **The Tower of London**: A historic fortress that has served as a palace, prison, armo

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

Tavily_search=TavilySearchResults()

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

def get_hotel(Location:str)->str:
       
    
    context=Tavily_search.invoke(f"list of top five (means best ratings) hotels in given location : {Location} along with their price per night")
    
    print(context)
    print("********************************************")
    Prompt_template=ChatPromptTemplate.from_template(
        """You are a helpful assistant. From this search result :\n\n{context}\n\n
        list the best 3 hotels in {Location} from given location within budget """   )
    
    chain=LLMChain(llm=search_llm,prompt=Prompt_template)
    result=chain.invoke({"context":context,"Location":Location})
    print(result["text"])
  
    pass 



In [44]:
get_hotel(Location="London")

[{'title': 'Luxury London hotels: The best five star stays in the capital in 2025', 'url': 'https://luxurylondon.co.uk/travel/london/best-five-star-hotels-in-london/', 'content': '*   Cookie\n    \n    \\_\\_cf\\_bm\n    \n*   Duration\n    \n    1 hour\n    \n*   Description\n    \n    This cookie, set by Cloudflare, is used to support Cloudflare Bot Management. [...] link](https://cdn-cookieyes.com/assets/images/external-link.svg)](https://asklocala.com/advertising-privacy-policy/?_gl=1*145zs89*_up*MQ..*_ga*MjA4NDgxNDYxOC4xNjk2NTEyNTU5*_ga_VRM83PCHBL*MTY5NjUxMjU1OC4xLjEuMTY5NjUxMjU3NS4wLjAuMA..*_ga_5N7836YC9X*MTY5NjUxMjU1OC4xLjEuMTY5NjUxMjU3NS4wLjAuMA..) [...] link](https://cdn-cookieyes.com/assets/images/external-link.svg)](https://asklocala.com/advertising-privacy-policy/?_gl=1*145zs89*_up*MQ..*_ga*MjA4NDgxNDYxOC4xNjk2NTEyNTU5*_ga_VRM83PCHBL*MTY5NjUxMjU1OC4xLjEuMTY5NjUxMjU3NS4wLjAuMA..*_ga_5N7836YC9X*MTY5NjUxMjU1OC4xLjEuMTY5NjUxMjU3NS4wLjAuMA..)', 'score': 0.50476116}]
************

In [46]:
Tavily_search.invoke(f"Top hotels in london along with their price per night")

[{'title': 'The best cheap London hotels from $59/night - Skyscanner',
  'url': 'https://www.skyscanner.com/hotels/united-kingdom/london-hotels/ci-27544008',
  'content': 'Best hotels in London based on location\u200b\u200b For the best-located London hotels, consider the Park Plaza London Waterloo, The Tower Hotel, and Hilton London',
  'score': 0.82391036},
 {'title': '16 Best Hotels in London. Hotels from $20/night - KAYAK',
  'url': 'https://www.kayak.com/London-Hotels.28501.hotel.ksp',
  'content': '![King Solomon Hotel- Golders Green - London - Bedroom](https://content.r9cdn.net/rimg/himg/59/e6/70/expedia_group-54913-e5e213-606328.jpg?width=335&height=268&crop=true)\n![The Z Hotel Covent Garden - London - Balcony](https://content.r9cdn.net/rimg/himg/9a/e6/69/expedia_group-4594486-62537031-328724.jpg?width=335&height=268&crop=true)\n![Hilton London Metropole - London - Bar](https://content.r9cdn.net/rimg/himg/7c/a2/10/leonardo-12285-194237892-880939.jpg?width=335&height=268&crop=t

In [80]:
def get_hotel1(Hotel_name:str,Location:str)->str:
       
    
    context=Tavily_search.invoke(f"provide {Hotel_name}, {Location} price per night")
    
    print(context)
    print("********************************************")
    Prompt_template=ChatPromptTemplate.from_template(
        """You are a helpful assistant. From this search result :\n\n{context}\n\n
        analyse and just extract Hotel name and its lotest price per night in USD """)
    
    chain=LLMChain(llm=search_llm,prompt=Prompt_template)
    result=chain.invoke({"context":context,"Location":Location})
    print(result["text"])

In [81]:
get_hotel1( "The Caesar Hotel","London")

[{'title': 'The Caesar Hotel in London, the United Kingdom from $123', 'url': 'https://www.momondo.com/hotels/london/Caesar-Hotel.mhd35932.ksp', 'content': '#### Distance to airport\n\n10.0 mi\n\n#### Check in/Check out\n\nCheck in anytime after 2:00 PM, check out anytime before 12:00 PM\n\n#### Cancellation/prepayment\n\nCancellation/prepayment policies vary by room type and provider.\n\n## Frequently asked questions\n\n#### How does The Caesar Hotel compare to other 4 star hotels in London?\n\nThere is a $205/night difference in the average price of 4 star hotels in London ($402/night) and the average nightly rate at The Caesar Hotel ($196/night). [...] ![Hero photo](/rimg/himg/a8/2e/77/expedia_group-35932-186204814-144255.jpg?width=1366&height=768&crop=true)\n\n# The Caesar Hotel\n\n4-star Hotel with Fitness center\n\n26-33 Queens Gardens, London, W2 3BE\n\nbased on 3,250 reviews\n\nper night\n\n![Mirai]()\n![Agoda.com]()\n![Agoda.com]()\n\n## About The Caesar Hotel, London\n\n## Th

In [72]:


from langchain_community.tools import DuckDuckGoSearchResults
duck_search=DuckDuckGoSearchResults()
print(duck_search.invoke("list top 5 best hotels in london?"))

snippet: Best hotel in London for: a London landmark hit-list in complete luxury. Design notes: This grande dame hotel dates back to 1887 and is all about elegant opulence. The Mandarin Oriental Hyde Park is one of London's most famous and most lavish places to stay. ... Our room was perched up the top of the hotel with a framed view of Victorian ..., title: The best hotels in London for 2025, tried and tested, link: https://www.houseandgarden.co.uk/gallery/best-hotels-in-london, snippet: Situated in the heart of London's elegant Mayfair area, Flemings Mayfair was voted as the best hotel in London by Travel + Leisure readers in the 2024 World's Best Awards. The 129-room hotel ..., title: The Best Hotels in London - Travel + Leisure, link: https://www.travelandleisure.com/best-hotels-in-london-8608254, snippet: Unforgettable food. Cutting-edge design. Immersive experiences. These are some of your top criteria for judging what makes a quality hotel—not to mention a worthy resort, cruise,

In [66]:

def get_hotel_data(Hotel_name:str,Location:str)->str:
    """This tool provides real-time hotel price per night"""
    result_duck=duck_search.invoke(f"provide {Hotel_name} hotel {Location} price per night  ")
    result_tavily=Tavily_search.invoke(f"provide {Hotel_name} hotel {Location} price per night")
    return result_duck,result_tavily

In [68]:
res_d,res_t=get_hotel_data("The Caesar Hotel","London")



res_d

'snippet: The Caesar Hotel Review. Offering a sauna, a gym and a meeting room, the comfortable Caesar Hotel London is located not far from Bayswater. Mayfair is 2.3 km from the venue and Paddington Street Gardens is 2.4 km away. Each morning a full English breakfast is available at the in the restaurant., title: The Caesar Hotel London with Minimum Price 48£: Expert Review | UPDATED ..., link: https://bedandbreakfast-london.com/caesar-hotel-london/, snippet: Book The Caesar Hotel, London, England on Tripadvisor: See 111 traveler reviews, 123 candid photos, and great deals for The Caesar Hotel, ranked #139 of 1,170 hotels in London, England and rated 4 of 5 at Tripadvisor. ... per night. $350. per night. $222. per night. Jun. Jul. Jun 10. Today. Jun 12. Jun 14. Jun 16. Jun 18. Jun 20. Jun 22. Jun 24 ..., title: THE CAESAR HOTEL - Updated 2025 Prices & Reviews (London, England), link: https://www.tripadvisor.com/Hotel_Review-g186338-d192097-Reviews-or20-The_Caesar_Hotel-London_England.ht

In [69]:
res_t

[{'title': 'The Caesar Hotel in London, the United Kingdom from $123',
  'url': 'https://www.momondo.com/hotels/london/Caesar-Hotel.mhd35932.ksp',
  'content': '#### Distance to airport\n\n10.0 mi\n\n#### Check in/Check out\n\nCheck in anytime after 2:00 PM, check out anytime before 12:00 PM\n\n#### Cancellation/prepayment\n\nCancellation/prepayment policies vary by room type and provider.\n\n## Frequently asked questions\n\n#### How does The Caesar Hotel compare to other 4 star hotels in London?\n\nThere is a $205/night difference in the average price of 4 star hotels in London ($402/night) and the average nightly rate at The Caesar Hotel ($196/night). [...] ![Hero photo](/rimg/himg/a8/2e/77/expedia_group-35932-186204814-144255.jpg?width=1366&height=768&crop=true)\n\n# The Caesar Hotel\n\n4-star Hotel with Fitness center\n\n26-33 Queens Gardens, London, W2 3BE\n\nbased on 3,250 reviews\n\nper night\n\n![Mirai]()\n![Agoda.com]()\n![Agoda.com]()\n\n## About The Caesar Hotel, London\n\n#

In [62]:
def calculate_budget():
    

TypeError: string indices must be integers, not 'str'